# Progetto Cognitive Computing & Artificial Intelligence

Studenti: *Angelo Frasca, Nunzio Fornitto, Fernando Riccioli*

## Introduzione
È stato realizzato un agente per la creazione di post per un blog di calcio. L'agente realizza una bozza di articolo con i pronostici per la prossima giornata della Serie A e formula tre idee per post futuri. L'agente è stato realizzato tramite LangGraph e implementa quattro nodi: caricamento dei dati, generazione della bozza, riflessione e valutazione.


## Flusso dell'Agente
- **Nel nodo di caricamento dei dati**, l'agente utilizza dei tool per ottenere i dati della Serie A da API. I tool sono implementati tramite decorator e vengono chiamati in maniera statica tramite `nome_tool.invoke("descrizione")`. Tutti i tool comprendono docstring, un parametro di input e il type int per l'output. Si è preferito utilizzare i tool invocati staticamente per rimanere coerenti con il flusso di funzionamento del programma. I dati che si ottengono tramite i tool sono fondamentali per l'esecuzione del programma e quindi si è preferito questo metodo piuttosto dell'utilizzo del binding.


- **Nel nodo di generazione** della bozza vengono anche generate le tre idee per i post futuri. Il modello viene chiamato utilizzando un prompt che implementa alcune tecniche di prompting avanzate come il role-prompting, il chain-of-thought e il constraint-prompting.

- **Nel nodo di reflection**, il modello riceve la bozza generata dal nodo precedente e fornisce una critica dettagliata. Anche questa critica viene generata usando delle tecniche di prompting avanzate che sono commentate esplicitamente nel codice (ad esempio l'adversarial-prompting).  Infine la critica viene inserita nello stato e l'agente torna nuovamente al nodo di generazione. Il nodo di generazione, infatti, comprende anche l'uso di un secondo prompt che viene chiamato durante il ciclo di reflection. In particolare il modello utilizza il commento critico ottenuto per migliorare la bozza precedente. Il numero di iterazioni del ciclo di reflection è modificabile tramite i multipli di due all'interno della funzione di routing `should_continue`. Con un parametro pari a 2, il ciclo viene eseguito una volta. In particolare, dopo che il nodo di generazione è stato utilizzato per migliorare la bozza si passa al nodo di evaluation.

- **Nel nodo di evaluation** il modello verifica che la bozza sia adatta alla pubblicazione. Il modello controlla se le linee guida specifiche nel prompt di questo nodo vengono rispettate. Se il post supera la valutazione termina il flusso dell'agente. Altrimenti l'agente torna nuovamente al nodo di generation, viene resettato il ciclo di reflection precedente e comincia da capo. Ciò significa che si tornerà nuovamente nel ciclo di reflection.

## Retrieval-Augmented Generation
È stata utilizzato anche la Retrieval Augmented Generation tramite un database vettoriale. I documenti presi dal web e trasformati in embedding riguardano metodi per generazione accurata di pronostici calcistici.

---
# Installazioni e Import

In [1]:
!pip install langchain langchain-community langchain-openai
!pip install beautifulsoup4 requests chromadb selenium
!pip install openai unstructured langgraph grandalf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 108.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.1 MB/s eta 0:00:00
 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.1/192.1 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 14.7 MB/s eta 0:00:00
   ━━

In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import SeleniumURLLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langgraph.graph import StateGraph
from langchain.schema import HumanMessage
from typing import TypedDict, List, Optional, Literal, Dict
from IPython.display import Image, display, HTML
from langchain_core.tools import tool
import requests, datetime, os

---
# Recupero dati tramite API

I dati recuperati tramite API sono essenziali per migliorare la qualità delle risposte generate dal modello.
- La classifica attuale della Serie A (punteggio, vittorie e sconfitte per ogni squadra)
- Le giornate del campionato (data e ora di ciascuna partita)
- I marcatori (giocatori con più gol)
- La giornata imminente (gruppo di partite del prossimo weekend)

Ad ogni tool è stato anche associato un esempio di utilizzo con recupero e stampa dei dati. Questo permette di verificare i dati durante l'esecuzione passo dopo passo.

In [2]:
@tool
def get_classifica(input) -> List[Dict[str, str]]:
    """Prendi la classifica della Serie A da API"""
    print(input)

    headers = {'X-Auth-Token': '6b588976b9ed413883530b28d5bd19cd'}
    url = 'https://api.football-data.org/v4/competitions/SA/standings'
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        standings = response.json() #Dizionario con la classifica
        squadre_info = []           #Lista di dizionari

        #Standing è un dizionario con le statistiche di un team
        for standing in standings['standings'][0]['table']:
            squadre_info.append({
                "squadra": standing['team']['name'],
                "PG": standing['playedGames'],
                "V": standing['won'],
                "N": standing['draw'],
                "P": standing['lost'],
                "GF": standing['goalsFor'],
                "GA": standing['goalsAgainst'],
                "DR": standing['goalsFor'] - standing['goalsAgainst'],
                "PT": standing['points']
            })
        return squadre_info
    else:
        raise Exception(f"Errore nella richiesta: {response.status_code}")

#Esempio di utilizzo
squadre_info = get_classifica.invoke("Prendo la classifica " +
                                      "della Serie A da API")
for squadra in squadre_info:
    print(squadra)

Prendo la classifica della Serie A da API
{'squadra': 'SSC Napoli', 'PG': 36, 'V': 23, 'N': 9, 'P': 4, 'GF': 57, 'GA': 27, 'DR': 30, 'PT': 78}
{'squadra': 'FC Internazionale Milano', 'PG': 36, 'V': 23, 'N': 8, 'P': 5, 'GF': 75, 'GA': 33, 'DR': 42, 'PT': 77}
{'squadra': 'Atalanta BC', 'PG': 36, 'V': 21, 'N': 8, 'P': 7, 'GF': 73, 'GA': 32, 'DR': 41, 'PT': 71}
{'squadra': 'Juventus FC', 'PG': 36, 'V': 16, 'N': 16, 'P': 4, 'GF': 53, 'GA': 33, 'DR': 20, 'PT': 64}
{'squadra': 'SS Lazio', 'PG': 36, 'V': 18, 'N': 10, 'P': 8, 'GF': 59, 'GA': 46, 'DR': 13, 'PT': 64}
{'squadra': 'AS Roma', 'PG': 36, 'V': 18, 'N': 9, 'P': 9, 'GF': 51, 'GA': 34, 'DR': 17, 'PT': 63}
{'squadra': 'Bologna FC 1909', 'PG': 36, 'V': 16, 'N': 14, 'P': 6, 'GF': 54, 'GA': 41, 'DR': 13, 'PT': 62}
{'squadra': 'AC Milan', 'PG': 36, 'V': 17, 'N': 9, 'P': 10, 'GF': 58, 'GA': 40, 'DR': 18, 'PT': 60}
{'squadra': 'ACF Fiorentina', 'PG': 36, 'V': 17, 'N': 8, 'P': 11, 'GF': 54, 'GA': 37, 'DR': 17, 'PT': 59}
{'squadra': 'Como 1907', '

In [3]:
@tool
def get_giornate(input) -> Dict[str, List]:
    """Prendi le partite della prossima giornata da API"""
    print(input)

    headers = {'X-Auth-Token': '6b588976b9ed413883530b28d5bd19cd'}
    url = 'https://api.football-data.org/v4/competitions/SA/matches'
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        matches = data['matches']
        matchdays = {}

        for match in matches:
            matchday = match['matchday']
            home_team = match['homeTeam']['name']
            away_team = match['awayTeam']['name']
            home_score = match['score']['fullTime']['home']
            away_score = match['score']['fullTime']['away']
            date = match['utcDate']
            if matchday not in matchdays:
                matchdays[matchday] = []

            matchdays[matchday].append({
                'home_team': home_team,
                'away_team': away_team,
                'home_score': home_score,
                'away_score': away_score,
                'date': date
            })
        return matchdays

#Esempio di utilizzo
matchdays = get_giornate.invoke("Prendo le partite della " +
                                "prossima giornata da API")
for matchday, games in matchdays.items():
    print(f"Giornata {matchday}:")
    for game in games:
        print(f"  {game['home_team']} {game['home_score']} - " +
        f"{game['away_score']} {game['away_team']} (Data:{game['date']})")
    print()

Prendo le partite della prossima giornata da API
Giornata 1:
  Genoa CFC 2 - 2 FC Internazionale Milano (Data:2024-08-17T16:30:00Z)
  Parma Calcio 1913 1 - 1 ACF Fiorentina (Data:2024-08-17T16:30:00Z)
  Empoli FC 0 - 0 AC Monza (Data:2024-08-17T18:45:00Z)
  AC Milan 2 - 2 Torino FC (Data:2024-08-17T18:45:00Z)
  Bologna FC 1909 1 - 1 Udinese Calcio (Data:2024-08-18T16:30:00Z)
  Hellas Verona FC 3 - 0 SSC Napoli (Data:2024-08-18T16:30:00Z)
  Cagliari Calcio 0 - 0 AS Roma (Data:2024-08-18T18:45:00Z)
  SS Lazio 3 - 1 Venezia FC (Data:2024-08-18T18:45:00Z)
  US Lecce 0 - 4 Atalanta BC (Data:2024-08-19T16:30:00Z)
  Juventus FC 3 - 0 Como 1907 (Data:2024-08-19T18:45:00Z)

Giornata 2:
  Parma Calcio 1913 2 - 1 AC Milan (Data:2024-08-24T16:30:00Z)
  Udinese Calcio 2 - 1 SS Lazio (Data:2024-08-24T16:30:00Z)
  FC Internazionale Milano 2 - 0 US Lecce (Data:2024-08-24T18:45:00Z)
  AC Monza 0 - 1 Genoa CFC (Data:2024-08-24T18:45:00Z)
  ACF Fiorentina 0 - 0 Venezia FC (Data:2024-08-25T16:30:00Z)
  To

In [4]:
@tool
def get_marcatori(input) -> List[Dict[str, str]]:
    """Prendi i marcatori della Serie A da API"""
    print(input)

    headers = {"X-Auth-Token": "6b588976b9ed413883530b28d5bd19cd"}
    url = "https://api.football-data.org/v4/competitions/SA/scorers"
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        dati = response.json()
        top_scorers = []
        for scorer in dati['scorers']:  #Dizionario di dizionari
            name = scorer['player']['name']
            team = scorer['team']['name']
            goals = scorer['goals']
            assists = scorer.get('assists', 0)
            top_scorers.append({        #Lista di dizionari
                'Giocatore': name,
                'Squadra': team,
                'Gol': goals,
                'Assist': assists
            })
        return top_scorers
    else:
        raise Exception(f"""Errore nella richiesta:
        {response.status_code}\nMessaggio: {response.text}""")

#Esempio di utilizzo
top_scorers = get_marcatori.invoke("Prendo i marcatori della " +
                                   "Serie A da API")
for scorer in top_scorers:
    print(f"{scorer['Giocatore']} - {scorer['Squadra']}: " +
    f"{scorer['Gol']} gol, {scorer['Assist']} assist")

Prendo i marcatori della Serie A da API
Mateo Retegui - Atalanta BC: 24 gol, 6 assist
Moise Kean - ACF Fiorentina: 17 gol, 2 assist
Ademola Lookman - Atalanta BC: 15 gol, 4 assist
Marcus Thuram-Ulien - FC Internazionale Milano: 14 gol, 5 assist
Riccardo Orsolini - Bologna FC 1909: 13 gol, 3 assist
Romelu Lukaku - SSC Napoli: 13 gol, 10 assist
Artem Dovbyk - AS Roma: 12 gol, 3 assist
Lautaro Martínez - FC Internazionale Milano: 12 gol, 3 assist
Christian Pulisic - AC Milan: 11 gol, 9 assist
Lorenzo Lucca - Udinese Calcio: 11 gol, 1 assist


In [5]:
@tool
def get_giornata_imminente(input) -> tuple[Optional[int], Optional[datetime.date], List]:
    """Prendo la prossima giornata della Serie A da API"""
    print(input)

    headers = {'X-Auth-Token': '6b588976b9ed413883530b28d5bd19cd'}
    url = 'https://api.football-data.org/v4/competitions/SA/matches'
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        matches = data['matches']
        matchdays = {}
        for match in matches:
            matchday = match['matchday']
            home_team = match['homeTeam']['name']
            away_team = match['awayTeam']['name']
            home_score = match['score']['fullTime']['home']
            away_score = match['score']['fullTime']['away']
            date = match['utcDate']
            #Se la partita non è stata giocata
            if home_score is None and away_score is None:
                if matchday not in matchdays:
                    matchdays[matchday] = []
                #Matchdays è un dizionario che contiene i match futuri
                matchdays[matchday].append({
                    'home_team': home_team,
                    'away_team': away_team,
                    'date': date
                })

        today = datetime.date.today()
        upcoming_matchdays = []
        for matchday, games in matchdays.items():
            first_match_date = min(games, key=lambda x: x['date'])['date']
            first_match_date = datetime.datetime.strptime(first_match_date,
                               "%Y-%m-%dT%H:%M:%SZ").date()
            if first_match_date >= today:
                upcoming_matchdays.append((first_match_date, matchday, games))
        upcoming_matchdays.sort()
        if upcoming_matchdays:
            matchday_date, matchday_num, games = upcoming_matchdays[0]
            return matchday_num, matchday_date, games
        else:
            return None, None, []
    else:
        raise Exception(f"Errore nella richiesta: {response.status_code}" +
                        f"\nMessaggio: {response.text}")

# Esempio di utilizzo
matchday_num, matchday_date, games = get_giornata_imminente.invoke("Prendo la prossima giornata della Serie A da API")
if games:
    print(f"Giornata {matchday_num} ({matchday_date}):")
    for game in games:
        print(f"  {game['home_team']} vs {game['away_team']} " +
        f"(Data: {game['date']})")
else:
    print("Nessuna partita futura disponibile.")

Prendo la prossima giornata della Serie A da API
Giornata 37 (2025-05-17):
  Genoa CFC vs Atalanta BC (Data: 2025-05-17T18:45:00Z)
  Cagliari Calcio vs Venezia FC (Data: 2025-05-18T18:45:00Z)
  ACF Fiorentina vs Bologna FC 1909 (Data: 2025-05-18T18:45:00Z)
  Hellas Verona FC vs Como 1907 (Data: 2025-05-18T18:45:00Z)
  FC Internazionale Milano vs SS Lazio (Data: 2025-05-18T18:45:00Z)
  Juventus FC vs Udinese Calcio (Data: 2025-05-18T18:45:00Z)
  US Lecce vs Torino FC (Data: 2025-05-18T18:45:00Z)
  AC Monza vs Empoli FC (Data: 2025-05-18T18:45:00Z)
  Parma Calcio 1913 vs SSC Napoli (Data: 2025-05-18T18:45:00Z)
  AS Roma vs AC Milan (Data: 2025-05-18T18:45:00Z)


---
# Retriever per la Retrieval-Augmented-Generation
Viene creato un retriever che utilizza dei post trovati sul web come linee guida per eseguire pronostici accurati. È stata utilizzata la libreria Selenium per il caricamento degli articoli da URL e Chroma per la creazione del database vettoriale.
Il retriever viene chiamato successivamente, nel nodo di caricamento dei dati. I dati vengono caricati nello stato e vengono utilizzati dal modello durante la fase di generazione.

In [6]:
def get_retriever():
    print("Caricamento dati per il Retriever...")

    urls = [
        "https://www.protipster.it/betting-news/consigli-per-" +
        "pronostici-vincenti-studiare-una-partita-di-calcio-4843",
        "https://topscommesse.com/come-pronosticare-le-partite-di-calcio/"
    ]
    loader = SeleniumURLLoader(urls=urls)
    docs = loader.load()

    # Split del testo in chunks
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=250, chunk_overlap=0
    )
    doc_splits = text_splitter.split_documents(docs)

    db_path = os.path.join(os.getcwd(), "chroma_db")
    if not os.path.exists(db_path):
        os.makedirs(db_path)

    # Creazione del database vettoriale
    vectorstore = Chroma.from_documents(
        documents=doc_splits,
        collection_name="rag-chroma",
        embedding=OpenAIEmbeddings(),
        persist_directory=db_path
    )
    return vectorstore.as_retriever()

---
#Definizioni del modello e dello stato

È stato utilizzato il modello GPT-4o-mini come LLM ed è stata associata una chiave per l'utilizzo modello.

In [ ]:
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini-2024-07-18")

In [8]:
class AgentState(TypedDict):
    match_data: str
    evaluation: str
    documents: List[str]
    messages: List[str]

- `match_data` contiene i dati recuperati tramite i tool
- `evaluation` contiene il riscontro (positivo o negativo) generato in fase di Valutazione
- `documents` contiene i documenti per la Retrieval-Augmented-Generation
- `messages` contiene i messaggi prodotti dal nodo di generazione e dal nodo di riflessione, che vengono usati per il ciclo di reflection

---
# Nodi del grafo

Il nodo di caricamento dei dati comprende l'invocazione statica dei tool per riempire ilcampo `match_data` dell'`AgentState`. Viene anche utilizzato il Retriever per riempire il campo `documents`.

In [9]:
#Nodo di Caricamento dei Dati
def load_data_node(state):
    print("\n--- CARICAMENTO DATI ---\n")

    classifica = get_classifica.invoke("Prendo la classifica della Serie A da API...")
    marcatori = get_marcatori.invoke("Prendo i marcatori della Serie A da API...")
    giornate = get_giornate.invoke("Prendo le giornate della Serie A da API...")
    giornata_imminente = get_giornata_imminente.invoke("Prendo la prossima giornata della Serie A da API....")

    try:
        match_data = {
            "giornata_imminente": giornata_imminente,
            "giornate": giornate,
            "marcatori": marcatori,
            "classifica": classifica
        }
        state["match_data"] = match_data

        #Chiamata al Retriever (RAG)
        retriever = get_retriever()
        documents = retriever.invoke("pronostico calcio")
        state["documents"] = documents

    except Exception:
        state["match_data"] = {}
        print("Errore durante lo scraping:", str(Exception))

    finally:
      return state

Il nodo di Generazione utilizza due prompt distinti. Il primo viene utilizzato quando il nodo è all'inizio del ciclo di generazione. Questo produce una bozza che poi viene inviata al nodo di Riflessione per essere valutato.  Il secondo prompt viene utilizzato, invece, quando il nodo ha ricevuto una critica costruttiva da parte del nodo di riflessione e per tutte le successive volte in cui viene chiamato questo nodo durante il ciclo. In entrambi i casi la risposta generata dal modello viene inserita come ultimo messaggio all'interno della lista `messages` dell'`AgentState`.

In [13]:
#Nodo di Generazione
def generation_node(state):
    print("\n--- GENERAZIONE ---\n")

    match_data = state["match_data"]
    giornata_imminente = match_data.get("giornata_imminente", [])
    if not giornata_imminente:
        print("Nessuna partita trovata per la prossima giornata.")
        return state
    numero_giornata = giornata_imminente[0]
    data_giornata = giornata_imminente[1]
    prossima_giornata = giornata_imminente[2]


    #Se è la prima generazione nel ciclo di reflection
    if state["messages"].__len__() == 0:
      #Tecniche di prompting utilizzate: role-prompting, retrieval-augmented
      #generation (Linee guida sulla creazione di un buon pronostico),
      #contextual-prompting, constrained-prompting
      prompt = f"""
      Sei un giornalista sportivo esperto in pronostici sulle partite
      di calcio.

      Conosci i seguenti dati:
      - Numero della prossima giornata: {numero_giornata}
      - Data della prossima giornata: {data_giornata}
      - Prossime partite: {prossima_giornata}
      - Classifica attuale della Serie A: {match_data["classifica"]}
      - Giornate precedenti: {match_data["giornate"]}
      - Statistiche su marcatori e assist: {match_data["marcatori"]}
      - Linee guida sulla creazione di un buon
        pronostico: {state["documents"]}

      In base a questi dati, per tutte le partite della prossima giornata
      Genera un'analisi, con una breve motivazione, considerando:
        - Forma recente dei giocatori e delle squadre
        - Posizione attuale in classifica
        - Marcatori in forma
        - Fattore campo
        - Pattern che sei in grado di trovare con la tua esperienza
        con formato di output seguente:
          1. Squadra A vs Squadra B → Pronostico: X → Motivazione: ... → Dettagli: ...

      Scrivi la risposta senza titoli, senza grassetto e senza righe
      vuote tra una riga di testo e l'altra.

      E genera tre idee per diversi post da pubblicare sui social
      media su queste partite imminenti.
      """

    #Se non è la prima generazione nel ciclo di reflection
    else:
      prompt = f"""
      Sei un giornalista sportivo esperto nei pronostici di calcio.

      Hai ricevuto una critica per migliorare un articolo che hai generato.
      Implementa le modifiche consigliate e produci un articolo aggiornato con
      i pronostici per le partite e tre idee per i post.

      Scrivi la risposta senza titoli, senza grassetto e senza righe
      vuote tra una riga di testo e l'altra.

      Ecco la critica sull'articolo precedente: {state["messages"][-1]}
      """

    response = llm.invoke([
        HumanMessage(content=prompt)
    ])
    prediction = response.content.strip()
    print(prediction)
    state["messages"].append(prediction)
    return state

Il nodo di Riflessione utilizza l'adversarial-prompting per fornire una critica costruttiva alla bozza generata.

In [14]:
#Nodo di Riflessione
def reflection_node(state):
    print("\n--- RIFLESSIONE ---\n")

    #Role-prompting, constrained-prompting,
    #adversarial prompting, chain-of-thought
    prompt = f"""
    Sei un giornalista sportivo esperto nella critica e nella revisione
    di articoli giornalistici.

    Rifletti sul seguente articolo e genera una critica dettagliata con
    una lista di consigli specifici su come migliorarlo. Spiega il tuo
    ragionamento passo dopo passo. Limita la critica e i consigli sul
    testo e la struttura dell'articolo. Prima della tua risposta fornisci
    anche il testo completo che hai ricevuto come input.

    Scrivi la risposta senza titoli, senza grassetto e senza righe
    vuote tra una riga di testo e l'altra. Rendi esplicito quando
    stai fornendo il testo che hai ricevuto come input e quando
    stai fornendo la critica.

    Testo: {state["messages"][-1]}
    """

    response = llm.invoke([HumanMessage(content=prompt)])
    improved = response.content.strip()
    print(improved)
    state["messages"].append(improved)
    return state

Il nodo di Valutazione fornisce le linee guida che devono essere rispettate affinché la bozza venga considerata adatta alla pubblicazione

In [15]:
#Nodo di Valutazione
def evaluate_node(state):
    print("\n--- VALUTAZIONE ---\n")

    #Role-prompting, constrained-prompting
    prompt = f"""
    Sei un giornalista sportivo esperto che lavora per un blog di
    notizie calcistiche. Hai ricevuto il seguente articolo sulle
    partite della prossima giornata: {state["messages"][-1]}

    Valuta se:
    - È presente una previsione per tutte le partite
    - Ogni previsione è accompagnata da una motivazione
    - Lo stile di scrittura è chiaro e professionale
    - Non ci sono contraddizioni
    - Sono presenti tre idee per dei post futuri da generare

    Rispondi solo con "True" se il contenuto è pronto per la
    pubblicazione, oppure "False" se necessita revisione.
    """

    response = llm.invoke([HumanMessage(content=prompt)])
    evaluation_result = response.content.strip().lower()
    is_valid = "true" in evaluation_result

    if is_valid:
      print("Valutazione: Contenuto accettato")
      state["evaluation"] = is_valid
    if not is_valid:
      print("Valutazione: Contenuto da rivedere")
      state["messages"].clear()

    return state

---
# Funzioni di Routing e creazione del Grafo

È possibile cambiare il numero di cicli di reflection modificando il parametro. Per eseguire un ciclo di reflection è necessario il `2`, per eseguire due cicli è necessario usare `4` e per eseguire tre cicli è necessario usare `6`. Questo perché `messages` è una lista che contiene in maniera alternata il messaggio del nodo di generazione e quello del nodo di riflessione. In un ciclo ciascun nodo produce un messaggio e quindi è necessario utilizzare multipli per modificare il numero di cicli.

In [16]:
def should_continue(state) -> Literal["evaluate", "reflect"]:
    if len(state["messages"]) > 2:  #È possibile aumentare il numero
                                    #di cicli di reflection cambiando
                                    #questo parametro (2, 4, 6, etc...)
        return "evaluate"
    else:
        return "reflect"

def route_from_evaluation(state) -> Literal["generate", "__end__"]:
    if state.get("evaluation") is True:
        return "__end__"
    else:
        state["messages"].clear()
        return "generate"

In [17]:
builder = StateGraph(AgentState)

builder.add_node("load_data", load_data_node)
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.add_node("evaluate", evaluate_node)

builder.add_conditional_edges("generate", should_continue)
builder.add_conditional_edges("evaluate", route_from_evaluation)

builder.set_entry_point("load_data")
builder.add_edge("load_data", "generate")
builder.add_edge("reflect", "generate")

graph = builder.compile()

# Visualizzazione del grafo
A causa di un bug riscontrato con la funzione `graph.get_graph()`, è stata creata la funzione `render_mermaid` per stampare il grafo tramite HTML, JavaScript e CDN. (Piuttosto che utilizzare `display(Image(graph.get_graph().draw_mermaid_png()))`).

In [18]:
from IPython.display import display, HTML

def render_mermaid(mermaid_code: str):
    html = f"""
    <style>.mermaid svg {{background-color: white ;}}</style>
    <div class="mermaid">{mermaid_code}</div>
    <script src="https://cdn.jsdelivr.net/npm/mermaid@8.13.10/dist/mermaid.min.js"></script>
    <script>
        if (typeof mermaid !== 'undefined') {{
            mermaid.initialize({{ startOnLoad: true }});
            mermaid.init(undefined, document.querySelectorAll(".mermaid"));
        }}
    </script>"""
    display(HTML(html))

mermaid_graph = """
  graph TD;
      __start__([<p>__start__</p>]):::first
      load_data(load_data)
      generate(generate)
      reflect(reflect)
      evaluate(evaluate)
      __end__([<p>__end__</p>]):::last
      __start__ --> load_data;
      evaluate -.-> generate;
      generate -.-> evaluate;
      generate -.-> reflect;
      load_data --> generate;
      reflect --> generate;
      evaluate -.-> __end__;"""

render_mermaid(mermaid_graph)

---
# Chiamata all'agente

In [19]:
initial_state = {"topic": "Serie A", "messages": []}
final_state = graph.invoke(initial_state)


--- CARICAMENTO DATI ---

Prendo la classifica della Serie A da API...
Prendo i marcatori della Serie A da API...
Prendo le giornate della Serie A da API...
Prendo la prossima giornata della Serie A da API....
Caricamento dati per il Retriever...

--- GENERAZIONE ---

1. Genoa CFC vs Atalanta BC → Pronostico: 2 → Motivazione: Atalanta ha mostrato una forma recente migliore, con 21 vittorie in 36 partite e un attacco prolifico, mentre il Genoa ha faticato, con solo 9 vittorie. → Dettagli: Atalanta ha un attacco forte, guidato da Mateo Retegui, che è in forma con 24 gol. Il fattore campo potrebbe non essere sufficiente per il Genoa, che ha una difesa vulnerabile.

2. Cagliari Calcio vs Venezia FC → Pronostico: 1 → Motivazione: Cagliari ha bisogno di punti per evitare la retrocessione e gioca in casa, dove ha una leggera vantaggio. Venezia ha avuto difficoltà in trasferta. → Dettagli: Cagliari ha 8 vittorie in 36 partite, ma la necessità di punti potrebbe motivarli a dare il massimo. 

3